In [1]:
%matplotlib qt
import pyoptflight as pof
from pyoptflight import initialize as optinit
from pyoptflight import plotting as optplot
import numpy as np
import concurrent.futures
import itertools

In [2]:
def solver_instance(N=50, init_opts={}) -> pof.Solver:
    kerbin = pof.Body('Kerbin')
    vehicle = pof.Stage.load_vehicle('mintoc_multi')
    config = pof.SolverConfig(
        T_min              = 0,
        T_max              = 1000,
        max_iter           = 1000,
        solver_tol         = 1e-4,
        T_init             = 100,
        integration_method = 'RK4',
        N                  = N,
    )
    x0 = pof.LatLngBound(
        lat  = 0,
        lng  = 0,
        alt  = 0,
        vel  = 1e-6,
        ERA0 = 0
    )
    xf = pof.KeplerianBound(
        i  = np.deg2rad(60),
        Ω  = np.deg2rad(15),
        ω  = 0,
        ha = 80,
        hp = 80,
    #    ν = np.deg2rad(100),
        body=kerbin
    )
    solver = pof.Solver(kerbin, vehicle, config, x0, xf)
    solver.create_nlp()
    # solver.initialize_from_func(pof.cubic_bezier_spline, init_opts)
    solver.initialize_from_func(pof.gravity_turn, init_opts)
    solver.solve_nlp()
    return solver

In [3]:
def thread_worker(params):
    # Call the solver_instance with specific arguments
    return solver_instance(**params)

In [4]:
def solve_multiple(params: dict) -> list[pof.Solver]:
   keys = list(params.keys())
   combinations = [dict(zip(keys, values)) for values in itertools.product(*params.values())]
   total = len(combinations)
   completed = 0
   responses = []

   # Use ThreadPoolExecutor to execute solver_instance calls concurrently
   with concurrent.futures.ThreadPoolExecutor(max_workers=total) as executor:
      future_to_params = {executor.submit(thread_worker, params): params for params in combinations}
      for future in concurrent.futures.as_completed(future_to_params):
         result = future.result()
         responses.append(result)
         completed += 1
         print(f"Progress: {completed}/{total} solves completed.")

   return responses

In [5]:
N_list = [10, 15, 20, 25, 50, 100, 200]
opts = [{}]
params = {'N': N_list, 'init_opts': opts}
responses = solve_multiple(params)

The minimum cost is 0.23768895344643354 rad
The minimum cost is 0.23768895344643354 rad
The minimum cost is 0.23768895344643354 rad
The minimum cost is 0.23768895344643354 rad
The minimum cost is 0.23768895344643354 rad
The minimum cost is 0.23768895344643354 rad
The minimum cost is 0.23768895344643354 rad
Progress: 1/7 solves completed.
Progress: 2/7 solves completed.
Progress: 3/7 solves completed.
Progress: 4/7 solves completed.
Progress: 5/7 solves completed.
Progress: 6/7 solves completed.
Progress: 7/7 solves completed.


In [8]:
total_execution_time = 0
for result in responses:
    stats = result.stats()
    print(f"Result N = {result.N[0]}")
    print(f"Status: {stats['status']}")
    print(f"Runtime: {stats['runtime']}")
    print(f"Final mass: {stats['final_mass']}\n")
    total_execution_time += stats['nlp_creation_time'] + stats['runtime']
print(f'Total execution_time: {total_execution_time//60}m {total_execution_time%60:.2f}s')
print(f'({total_execution_time:.2f}s)')

Result N = 20
Status: Solve_Succeeded
Runtime: 12.639557361602783
Final mass: 3.969586738815457

Result N = 100
Status: Solve_Succeeded
Runtime: 30.945773601531982
Final mass: 4.1572912917729825

Result N = 200
Status: Solve_Succeeded
Runtime: 192.16201949119568
Final mass: 2.7294142012237725

Result N = 25
Status: Solve_Succeeded
Runtime: 213.1412227153778
Final mass: 4.05236634065859

Result N = 50
Status: Solve_Succeeded
Runtime: 199.09173965454102
Final mass: 3.5680758102425965

Result N = 15
Status: Solve_Succeeded
Runtime: 210.46097803115845
Final mass: 4.1444260079037685

Result N = 10
Status: Solve_Succeeded
Runtime: 213.4140830039978
Final mass: 4.020863300047437

Total execution_time: 18.0m 13.82s
(1093.82s)
